In [22]:
#goal
#determine "must-have" amenities that result in a great customer experience and ultimately a good review score

#assumptions
#amenities are assumed to exist for all time review data despite the possibiliy of hosts adding/removing amenities over time

In [23]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

In [24]:
df = pd.read_csv(r'../data/2022-09-15/listings.csv')
df['amenities'] = df['amenities'].apply(lambda x: pd.eval(x)) #read_csv was importing 'amenities' as a string - this converts it to a series of lists
display(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7733 entries, 0 to 7732
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7733 non-null   int64  
 1   listing_url                                   7733 non-null   object 
 2   scrape_id                                     7733 non-null   int64  
 3   last_scraped                                  7733 non-null   object 
 4   source                                        7733 non-null   object 
 5   name                                          7733 non-null   object 
 6   description                                   7691 non-null   object 
 7   neighborhood_overview                         5502 non-null   object 
 8   picture_url                                   7733 non-null   object 
 9   host_id                                       7733 non-null   i

None

In [25]:
df = df[['id','review_scores_rating', 'amenities','number_of_reviews']]
display(df.info(verbose=True))
#review_score_rating has nulls, will filtering on number_of_review > 0 will remove them? If not we should figure out why they're null.
df = df[df.number_of_reviews > 0]
display(df.info(verbose=True))
#filter worked, and now we have no nulls (amenities list can still have nulls, will look into later)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7733 entries, 0 to 7732
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    7733 non-null   int64  
 1   review_scores_rating  6824 non-null   float64
 2   amenities             7733 non-null   object 
 3   number_of_reviews     7733 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 241.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6824 entries, 0 to 7732
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    6824 non-null   int64  
 1   review_scores_rating  6824 non-null   float64
 2   amenities             6824 non-null   object 
 3   number_of_reviews     6824 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 266.6+ KB


None

In [35]:
mlb = MultiLabelBinarizer()
amenities = pd.DataFrame(mlb.fit_transform(df['amenities']),columns=mlb.classes_, index=df.index)
print(amenities.shape)
display(amenities.head())
#appears to be thousands of amenities that can be custom entered by the host - we'll have to reduce this to the standard amenities airbnb provides

(6824, 2296)


,Beekman Fresh Air shampoo,GE stove,Small bluetooth speaker Bluetooth sound system,"""Everyone"" and a local non-profit bar of soap. body soap","""Everyone"" shampoo",1 burner portable induction glass stove top induction stove,1/2 size in kitchenette with separate freezer and mini fridge in one bedroom refrigerator,"100"" TV with Roku, Amazon Prime Video, Netflix, Apple TV, HBO Max","120"" HDTV with Apple TV, Netflix, premium cable, Amazon Prime Video","13"" TV",...,Yamaha Sound Bar Bluetooth sound system,Yuzu hand soap and hand lotion body soap,Zealios body soap,Zealios conditioner,Zealios shampoo,Zogics Citrus + Aloe Bodywash body soap,Zogics! shampoo,"Zogics: 3 in 1 (shampoo, body wash, hand soap): Citrus +Aloe body soap",Zogics: Citrus+Aloe shampoo,Zogics: Plant based conditioner infused with essential oils & botanical extracts conditioner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
print(amenities.info(verbose=True))
#also appears to be a string encoding issue. Will debug this later

Exception in callback BaseAsyncIOLoop._handle_events(1356, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(1356, 1)>
Traceback (most recent call last):
  File "c:\Users\Neo\AppData\Local\Programs\Python\Python310\lib\site-packages\jupyter_client\session.py", line 102, in json_packer
    ).encode("utf8")
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 25011-25012: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Neo\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\Neo\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\Neo\AppData\Local\Programs\Python\Python310\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv(